In [91]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner as kt
from sklearn.model_selection import train_test_split

In [117]:
def transform_data(df,index):
    data=df[index].to_numpy()
    new_data=np.zeros((len(data),max(data)+1))    
    for i in range(len(data)):
        new_data[i,data[i]]=1
    return new_data
    
df_spectra=pd.read_csv("spectrum_energy_input_numberOfPeaks.csv",index_col=[0])



x_data=transform_data(df_spectra,"no_of_max")
print(x_data[0:2])



y_data=df_spectra[["k6a1","k6a2","k11","k12","k9a1","k9a2"]].to_numpy()
y_data=y_data-0.5
y_data=y_data*4
print(y_data[0:10])


data_train, data_test, labels_train, labels_test = train_test_split(x_data, y_data, test_size=0.20, random_state=42)

print(data_train.shape)
print(labels_train.shape)

#df_training = df_spectra.sample(frac=0.8,random_state=10)
#df_testing=df_spectra.drop(df_training.index)
df_spectra.head()

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1.  0. -1.]
 [-1. -1. -1. -1.  0.  0.]
 [-1. -1. -1. -1.  0.  1.]
 [-1. -1. -1. -1.  1. -1.]
 [-1. -1. -1. -1.  1.  0.]
 [-1. -1. -1. -1.  1.  1.]
 [-1. -1. -1.  0. -1. -1.]]
(583, 23)
(583, 6)


,main_maximum,all_maxima,no_of_max,k6a1,k6a2,k11,k12,k9a1,k9a2,Energy_highest_max_normalized,Energy_all_max_normalized
0,0.748,[-0.758 -0.69 -0.612 -0.472 -0.406 -0.328 -0....,18,0.25,0.25,0.25,0.25,0.25,0.25,0.874,[0.121 0.155 0.194 0.264 0.297 0.336 0.424 0.4...
1,-0.574,[-0.574 -0.048 0.034 0.596 0.658],5,0.25,0.25,0.25,0.25,0.25,0.50,0.213,[0.213 0.476 0.517 0.798 0.829]
2,0.732,[-0.928 -0.776 -0.708 -0.628 -0.486 -0.422 -0....,19,0.25,0.25,0.25,0.25,0.25,0.75,0.866,[0.036 0.112 0.146 0.186 0.257 0.289 0.33 0.4...
3,0.734,[-0.926 -0.778 -0.712 -0.628 -0.486 -0.422 -0....,19,0.25,0.25,0.25,0.25,0.50,0.25,0.867,[0.037 0.111 0.144 0.186 0.257 0.289 0.33 0.4...
4,-0.560,[-0.56 -0.03 0.046 0.614 0.674],5,0.25,0.25,0.25,0.25,0.50,0.50,0.220,[0.22 0.485 0.523 0.807 0.837]


In [118]:
feature_columns_A = []

no_of_max = tf.feature_column.numeric_column("no_of_max")
no_of_max_bucket = tf.feature_column.bucketized_column(no_of_max,[i for i in range(22)])
my_feature_layer_A = tf.keras.layers.DenseFeatures(no_of_max_bucket)

#transform data


In [193]:

def plot_the_loss_curve(epochs, mse,val_mse):
    """Plot a curve of loss vs. epoch."""

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Squared Error")

    plt.plot(epochs, mse, label="Loss")
    plt.plot(epochs, val_mse, label="Val Loss")

    plt.legend()
    #plt.ylim([0, 15])
    plt.show()  

def create_model(my_learning_rate, my_feature_layer):
    model = tf.keras.models.Sequential()
    # Add the layer containing the feature columns to the model.
    # Define the first hidden layer with 10 nodes.   
   # model.add(my_feature_layer_A)
    layers=[5,10,10]
    #layers=[20000,3500,300,200,3000,30,20,30,30]
    for layer in layers:
        model.add(tf.keras.layers.Dense(units = layer, activation = 'relu',kernel_regularizer=tf.keras.regularizers.l1(0.04)))
    # Define the output layer.
    model.add(tf.keras.layers.Dense(units=6,  
                                    name='Output'))#,kernel_regularizer=tf.keras.regularizers.l2(0.04)))                              
  
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])

    return model



def train_model(model, x,y, epochs, label_name,
                batch_size=None):

    #features = {name:np.array(value) for name, value in dataset.items()}
    #label=dataset[label_name].to_numpy()
    history = model.fit(x=x, y=y, batch_size=batch_size,
                      epochs=epochs, shuffle=True, verbose=1,validation_split=0.2) 
    
    epochs = history.epoch
  
    df_hist = pd.DataFrame(history.history)
    #hist.head()
    #mse = hist["mean_squared_error"]
    mae = df_hist["mean_squared_error"].to_numpy()
    val_mae = df_hist["val_mean_squared_error"].to_numpy()
    return epochs,mae,val_mae

In [ ]:
# The following variables are the hyperparameters.
#data_train, data_test, labels_train, labels_test
learning_rate = 1e-2
epochs = 3000
batch_size = 400

label_name = ["k6a1","k6a2","k11","k12","k9a1","k9a2"]

my_model = create_model(learning_rate, my_feature_layer_A)

epochs, mse,val_mse = train_model(my_model, data_train,labels_train, epochs, 
                          label_name, batch_size)


Epoch 1/3000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2/2 [==============================] - ETA: 0s - loss: 3.3360 - mean_squared_error: 0.67 - 0s 36ms/step - loss: 3.3211 - mean_squared_error: 0.6718 - val_loss: 3.1472 - val_mean_squared_error: 0.6900
Epoch 2/3000
2/2 [==============================] - ETA: 0s - loss: 3.1256 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 3.1096 - mean_squared_error: 0.6666 - val_loss: 2.9516 - val_mean_squared_error: 0.6901
Epoch 3/3000
2/2 [==============================] - ETA: 0s - loss: 2.9268 - mean_s

2/2 [==============================] - ETA: 0s - loss: 0.7011 - mean_squared_error: 0.66 - 0s 12ms/step - loss: 0.6990 - mean_squared_error: 0.6652 - val_loss: 0.7255 - val_mean_squared_error: 0.6928
Epoch 37/3000
2/2 [==============================] - ETA: 0s - loss: 0.7036 - mean_squared_error: 0.67 - 0s 9ms/step - loss: 0.6979 - mean_squared_error: 0.6653 - val_loss: 0.7229 - val_mean_squared_error: 0.6927
Epoch 38/3000
2/2 [==============================] - ETA: 0s - loss: 0.6981 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6951 - mean_squared_error: 0.6652 - val_loss: 0.7177 - val_mean_squared_error: 0.6919
Epoch 39/3000
2/2 [==============================] - ETA: 0s - loss: 0.6907 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6906 - mean_squared_error: 0.6651 - val_loss: 0.7144 - val_mean_squared_error: 0.6908
Epoch 40/3000
2/2 [==============================] - ETA: 0s - loss: 0.6890 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6885 - mean_squared_error: 0.6650 

Epoch 75/3000
2/2 [==============================] - ETA: 0s - loss: 0.6793 - mean_squared_error: 0.66 - 0s 11ms/step - loss: 0.6788 - mean_squared_error: 0.6655 - val_loss: 0.7010 - val_mean_squared_error: 0.6875
Epoch 76/3000
2/2 [==============================] - ETA: 0s - loss: 0.6789 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6789 - mean_squared_error: 0.6654 - val_loss: 0.7013 - val_mean_squared_error: 0.6880
Epoch 77/3000
2/2 [==============================] - ETA: 0s - loss: 0.6769 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6785 - mean_squared_error: 0.6653 - val_loss: 0.7009 - val_mean_squared_error: 0.6889
Epoch 78/3000
2/2 [==============================] - ETA: 0s - loss: 0.6764 - mean_squared_error: 0.66 - 0s 8ms/step - loss: 0.6773 - mean_squared_error: 0.6653 - val_loss: 0.7025 - val_mean_squared_error: 0.6900
Epoch 79/3000
2/2 [==============================] - ETA: 0s - loss: 0.6744 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6777 - mean_squared_

2/2 [==============================] - ETA: 0s - loss: 0.6806 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6786 - mean_squared_error: 0.6652 - val_loss: 0.7021 - val_mean_squared_error: 0.6894
Epoch 114/3000
2/2 [==============================] - ETA: 0s - loss: 0.6768 - mean_squared_error: 0.66 - 0s 11ms/step - loss: 0.6781 - mean_squared_error: 0.6653 - val_loss: 0.7031 - val_mean_squared_error: 0.6895
Epoch 115/3000
2/2 [==============================] - ETA: 0s - loss: 0.6817 - mean_squared_error: 0.66 - 0s 8ms/step - loss: 0.6789 - mean_squared_error: 0.6654 - val_loss: 0.7014 - val_mean_squared_error: 0.6896
Epoch 116/3000
2/2 [==============================] - ETA: 0s - loss: 0.6773 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6772 - mean_squared_error: 0.6654 - val_loss: 0.7027 - val_mean_squared_error: 0.6899
Epoch 117/3000
2/2 [==============================] - ETA: 0s - loss: 0.6749 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6784 - mean_squared_error: 0.6

2/2 [==============================] - ETA: 0s - loss: 0.6837 - mean_squared_error: 0.67 - 0s 9ms/step - loss: 0.6777 - mean_squared_error: 0.6649 - val_loss: 0.7034 - val_mean_squared_error: 0.6901
Epoch 152/3000
2/2 [==============================] - ETA: 0s - loss: 0.6745 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6783 - mean_squared_error: 0.6650 - val_loss: 0.7032 - val_mean_squared_error: 0.6898
Epoch 153/3000
2/2 [==============================] - ETA: 0s - loss: 0.6770 - mean_squared_error: 0.66 - 0s 11ms/step - loss: 0.6782 - mean_squared_error: 0.6649 - val_loss: 0.7022 - val_mean_squared_error: 0.6898
Epoch 154/3000
2/2 [==============================] - ETA: 0s - loss: 0.6791 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6774 - mean_squared_error: 0.6649 - val_loss: 0.7039 - val_mean_squared_error: 0.6900
Epoch 155/3000
2/2 [==============================] - ETA: 0s - loss: 0.6782 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6787 - mean_squared_error: 0.6

...

2/2 [==============================] - ETA: 0s - loss: 0.6725 - mean_squared_error: 0.65 - 0s 9ms/step - loss: 0.6779 - mean_squared_error: 0.6651 - val_loss: 0.7067 - val_mean_squared_error: 0.6930
Epoch 380/3000
2/2 [==============================] - ETA: 0s - loss: 0.6775 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6788 - mean_squared_error: 0.6651 - val_loss: 0.7053 - val_mean_squared_error: 0.6926
Epoch 381/3000
2/2 [==============================] - ETA: 0s - loss: 0.6714 - mean_squared_error: 0.65 - 0s 8ms/step - loss: 0.6779 - mean_squared_error: 0.6652 - val_loss: 0.7046 - val_mean_squared_error: 0.6921
Epoch 382/3000
2/2 [==============================] - ETA: 0s - loss: 0.6790 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6779 - mean_squared_error: 0.6653 - val_loss: 0.7052 - val_mean_squared_error: 0.6918
Epoch 383/3000
2/2 [==============================] - ETA: 0s - loss: 0.6800 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6786 - mean_squared_error: 0.66

2/2 [==============================] - ETA: 0s - loss: 0.6805 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6786 - mean_squared_error: 0.6653 - val_loss: 0.7043 - val_mean_squared_error: 0.6907
Epoch 418/3000
2/2 [==============================] - ETA: 0s - loss: 0.6761 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6786 - mean_squared_error: 0.6652 - val_loss: 0.7032 - val_mean_squared_error: 0.6907
Epoch 419/3000
2/2 [==============================] - ETA: 0s - loss: 0.6745 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6776 - mean_squared_error: 0.6651 - val_loss: 0.7035 - val_mean_squared_error: 0.6906
Epoch 420/3000
2/2 [==============================] - ETA: 0s - loss: 0.6791 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6782 - mean_squared_error: 0.6651 - val_loss: 0.7034 - val_mean_squared_error: 0.6906
Epoch 421/3000
2/2 [==============================] - ETA: 0s - loss: 0.6723 - mean_squared_error: 0.65 - 0s 9ms/step - loss: 0.6777 - mean_squared_error: 0.66

2/2 [==============================] - ETA: 0s - loss: 0.6824 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6779 - mean_squared_error: 0.6651 - val_loss: 0.7027 - val_mean_squared_error: 0.6901
Epoch 456/3000
2/2 [==============================] - ETA: 0s - loss: 0.6777 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6779 - mean_squared_error: 0.6651 - val_loss: 0.7037 - val_mean_squared_error: 0.6897
Epoch 457/3000
2/2 [==============================] - ETA: 0s - loss: 0.6778 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6789 - mean_squared_error: 0.6651 - val_loss: 0.7016 - val_mean_squared_error: 0.6895
Epoch 458/3000
2/2 [==============================] - ETA: 0s - loss: 0.6777 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6771 - mean_squared_error: 0.6651 - val_loss: 0.7017 - val_mean_squared_error: 0.6894
Epoch 459/3000
2/2 [==============================] - ETA: 0s - loss: 0.6816 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6776 - mean_squared_error: 0.66

2/2 [==============================] - ETA: 0s - loss: 0.6758 - mean_squared_error: 0.66 - 0s 11ms/step - loss: 0.6793 - mean_squared_error: 0.6651 - val_loss: 0.7036 - val_mean_squared_error: 0.6918
Epoch 494/3000
2/2 [==============================] - ETA: 0s - loss: 0.6844 - mean_squared_error: 0.67 - 0s 9ms/step - loss: 0.6770 - mean_squared_error: 0.6650 - val_loss: 0.7057 - val_mean_squared_error: 0.6920
Epoch 495/3000
2/2 [==============================] - ETA: 0s - loss: 0.6793 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6788 - mean_squared_error: 0.6650 - val_loss: 0.7051 - val_mean_squared_error: 0.6919
Epoch 496/3000
2/2 [==============================] - ETA: 0s - loss: 0.6743 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6780 - mean_squared_error: 0.6650 - val_loss: 0.7045 - val_mean_squared_error: 0.6919
Epoch 497/3000
2/2 [==============================] - ETA: 0s - loss: 0.6743 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6776 - mean_squared_error: 0.6

2/2 [==============================] - ETA: 0s - loss: 0.6816 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6782 - mean_squared_error: 0.6651 - val_loss: 0.7025 - val_mean_squared_error: 0.6890
Epoch 532/3000
2/2 [==============================] - ETA: 0s - loss: 0.6789 - mean_squared_error: 0.66 - 0s 11ms/step - loss: 0.6786 - mean_squared_error: 0.6651 - val_loss: 0.7024 - val_mean_squared_error: 0.6891
Epoch 533/3000
2/2 [==============================] - ETA: 0s - loss: 0.6763 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6783 - mean_squared_error: 0.6650 - val_loss: 0.7023 - val_mean_squared_error: 0.6891
Epoch 534/3000
2/2 [==============================] - ETA: 0s - loss: 0.6726 - mean_squared_error: 0.65 - 0s 9ms/step - loss: 0.6782 - mean_squared_error: 0.6650 - val_loss: 0.7025 - val_mean_squared_error: 0.6892
Epoch 535/3000
2/2 [==============================] - ETA: 0s - loss: 0.6786 - mean_squared_error: 0.66 - 0s 9ms/step - loss: 0.6782 - mean_squared_error: 0.6

In [ ]:
features = {name:np.array(value) for name, value in df_testing.items()}
label=df_testing[label_name].to_numpy()
#data_train, data_test, labels_train, labels_test

plot_the_loss_curve(epochs, mse,val_mse)
evaluation=my_model.evaluate(x = data_test, y = labels_test, batch_size=batch_size)
predicted = my_model.predict(data_test)
print(evaluation)



df_test=pd.DataFrame(labels_test,columns=["k6a1_test","k6a2_test","k11_test","k12_test","k9a1_test","k9a2_test"])
df_predict=pd.DataFrame(predicted,columns=["k6a1_hat","k6a2_hat","k11_hat","k12_hat","k9a1_hat","k9a2_hat"])
pd.concat([df_test,df_predict], axis=1).head()